# Q1

El primer paso es seleccionar lo datos desde MongoDB, pasarlos a un dataframe, limpiarlos y guardar las seleccion de datos en un archivo json para volver a llevarlos a MongoDB.

In [31]:
# Tu codigo aqui
import pymongo
import pandas as pd

In [2]:
cliente=pymongo.MongoClient('mongodb://localhost:27017/')

In [7]:
db = cliente.db_companies
colec = db.companies

In [67]:
categories = list(colec.find({},{'category_code':1,'_id':0}))

In [65]:
set(list(map(lambda x:list(x.values())[0],categories)))

{None,
 'advertising',
 'analytics',
 'automotive',
 'biotech',
 'cleantech',
 'consulting',
 'design',
 'ecommerce',
 'education',
 'enterprise',
 'fashion',
 'finance',
 'games_video',
 'government',
 'hardware',
 'health',
 'hospitality',
 'legal',
 'local',
 'manufacturing',
 'medical',
 'messaging',
 'mobile',
 'music',
 'nanotech',
 'network_hosting',
 'news',
 'nonprofit',
 'other',
 'photo_video',
 'public_relations',
 'real_estate',
 'search',
 'security',
 'semiconductor',
 'social',
 'software',
 'sports',
 'transportation',
 'travel',
 'web'}

In [84]:
comp= list(colec.find({'$and':[
                          {'$or':[{
                          'category_code':'software'},
                          {'category_code':'games_video'},
                          {'category_code':'web'},
                          {'category_code':'ecommerce'},
                          {'category_code':'mobile'},
                          {'category_code':'photo_video'},
                          {'category_code':'network_hosting'}
                          ]},
                  {'number_of_employees':{'$gt':0}},
                  {'offices':{'$ne':[]}},
                  {'offices.latitude':{'$ne':[]}},
                  {'offices.longitude':{'$ne':[]}},
                  {'offices.latitude':{'$ne':None}},
                  {'offices.longitude':{'$ne':None}},
                  {'deadpooled_year':None},
                  {'number_of_employees':{'$ne':None}},
                  {'total_money_raised':{'$ne':None}}
                    ]},
                    {'_id':0,'name':1,'number_of_employees':1,'offices':1}
                    ).sort('number_of_employees',-1))

In [85]:
print(f'Whe are considering {len(comp)} offices')

Whe are considering 2927 offices


In [94]:
df1 = pd.DataFrame(comp).drop('offices',axis=1)
df1.head()

,name,number_of_employees
0,IBM,388000
1,Nokia,125000
2,Comcast,100000
3,MetaCarta,99999
4,Microsoft,90000


In [87]:
from pandas.io.json import json_normalize

offices = json_normalize(comp,'offices')
offices.head()

,address1,address2,city,country_code,description,latitude,longitude,state_code,zip_code
0,New Orchard Road,,Armonk,USA,Corporate Headquarters,41.109534,-73.723999,NY,10504
1,Keilalahdentie 2-4,"P.O. Box 226, FIN-00045 Nokia Group",,FIN,headquarter,60.170514,24.827224,None,FIN-00045
2,1701 JFK Blvd,,Philadelphia,USA,One Comcast Center,39.952576,-75.165252,PA,19103
3,350 Massachusetts Ave,,Cambridge,USA,World Headquarters,42.362360,-71.099692,MA,02139
4,1 Microsoft Way,,Redmond,USA,World Headquarters,47.643583,-122.128585,WA,


In [88]:
df2 = pd.DataFrame(offices)
df2.head()

,address1,address2,city,country_code,description,latitude,longitude,state_code,zip_code
0,New Orchard Road,,Armonk,USA,Corporate Headquarters,41.109534,-73.723999,NY,10504
1,Keilalahdentie 2-4,"P.O. Box 226, FIN-00045 Nokia Group",,FIN,headquarter,60.170514,24.827224,None,FIN-00045
2,1701 JFK Blvd,,Philadelphia,USA,One Comcast Center,39.952576,-75.165252,PA,19103
3,350 Massachusetts Ave,,Cambridge,USA,World Headquarters,42.362360,-71.099692,MA,02139
4,1 Microsoft Way,,Redmond,USA,World Headquarters,47.643583,-122.128585,WA,


In [103]:
final_df = pd.concat([df1, df2], axis=1, join='inner')
final_df['Geolocation'] = final_df.apply(lambda final_df:{'type':'Point', 'coordinates':[final_df.longitude, final_df.latitude]}, axis=1)
final_df.head()

,name,number_of_employees,address1,address2,city,country_code,description,latitude,longitude,state_code,zip_code,Geolocation
0,IBM,388000,New Orchard Road,,Armonk,USA,Corporate Headquarters,41.109534,-73.723999,NY,10504,"{'type': 'Point', 'coordinates': [-73.723999, ..."
1,Nokia,125000,Keilalahdentie 2-4,"P.O. Box 226, FIN-00045 Nokia Group",,FIN,headquarter,60.170514,24.827224,None,FIN-00045,"{'type': 'Point', 'coordinates': [24.827224, 6..."
2,Comcast,100000,1701 JFK Blvd,,Philadelphia,USA,One Comcast Center,39.952576,-75.165252,PA,19103,"{'type': 'Point', 'coordinates': [-75.1652518,..."
3,MetaCarta,99999,350 Massachusetts Ave,,Cambridge,USA,World Headquarters,42.362360,-71.099692,MA,02139,"{'type': 'Point', 'coordinates': [-71.099692, ..."
4,Microsoft,90000,1 Microsoft Way,,Redmond,USA,World Headquarters,47.643583,-122.128585,WA,,"{'type': 'Point', 'coordinates': [-122.128585,..."


In [104]:
final_df.to_json('geop.json', orient='records')